In [ ]:
#!pip install pyreadstat
#!pip install matplotlib seaborn
#!pip install mplcursors

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib
import mplcursors
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.rcParams['font.family']='Malgun Gothic'

In [ ]:
#데이터 불러오기
raw_welfare = pd.read_spss('D:\syb_data\data\Koweps_hpda18_2023_beta1.sav')
#데이터 불러오기
raw_welfare_19 = pd.read_spss('D:\syb_data\data\Koweps_hpwc14_2019_beta2.sav')
#복사본 만들기
welfare = raw_welfare.copy()
#복사본 만들기
welfare_19 = raw_welfare_19.copy()

In [ ]:
#데이터 검토
#welfare           # 앞부분, 뒷부분 확인
#welfare.shape     # 행, 열의 개수
#welfare.info()    # 열의 속성
#welfare.describe() # 요약통계

In [ ]:
# 분석에 필요한 열을 결정 // 열 이름 변경
welfare = welfare.rename(
    columns= {'h18_g3' : 'sex',                # 성별
              'h18_g4' : 'birth',              # 태어난 년도
              'h18_g10' : 'marriage_type',    # 혼인 상태
              'h18_g11' : 'religion',          # 종교
              'p1802_8aq1' : 'income',         # 월급
              'h18_eco9' : 'code_job',         # 직업 코드
              'h18_reg7' : 'code_region',      # 지역 코드
              'h18_eco4' : 'eW' ,              # 주된 경제활동 참여상태
              'h18_eco11': 'no_eW',              #비경제활동사유
              'h18_g8' : 'disabled_type', #장애종류
              'h18_med3' : 'hospital_year', #1년간 의료기간 이용 외래진료횟수
              'h18_soc_5' : 'pension_status' #국민연급 납부여부    
                              
              })    


In [ ]:
welfare_19 = welfare_19.rename(
    columns= {'h14_g3' : 'sex',                # 성별
              'h14_g4' : 'birth',              # 태어난 년도
              'h14_g10' : 'marriage_type',    # 혼인 상태
              'h14_g11' : 'religion',          # 종교
              'p1402_8aq1' : 'income',         # 월급
              'h14_eco9' : 'code_job',         # 직업 코드
              'h14_reg7' : 'code_region',       # 지역 코드
              'h14_eco4' : 'eW' ,              # 주된 경제활동 참여상태
              'h14_eco11' : 'no_eW',       # 비경제활동사유
              'h14_g8' : 'disabled_type', #장애종류
              'h14_med3' : 'hospital_year', #1년간 의료기간 이용 외래진료횟수
              'h14_soc_5' : 'pension_status', #국민연급 납부여부  
              })  

In [ ]:
welfare = welfare.assign(age = 2019 - welfare['birth']+1 )
welfare['age'].describe()

In [ ]:
#초년층 : 30세 미만, 중년층 : 30~59세 , 노년층 : 60세 이상

#나이 변수 확인

welfare['age'].head()

In [ ]:
welfare = welfare.assign(ageg = np.where(welfare['age']<30, 'young',
                                np.where(welfare['age']<=59, 'middle','old')))
welfare['ageg'].value_counts()

In [ ]:
# 빈도 그래프
sns.countplot(data=welfare, x= 'ageg')

### 연령대 별 취업률과 실업률 비율 

In [ ]:
welfare['eW'].describe()

In [ ]:
welfare['eW'].dtype
welfare['eW'].value_counts() # 빈도수 

In [ ]:
#이상치 결측 처리 

welfare['eW']=np.where(welfare['eW'] == 99, np.nan, welfare['eW'])


# 결측 확인
print("결측 확인:", welfare['eW'].isna().sum()) 
welfare['eW'].isna().sum()

In [ ]:
welfare=welfare.dropna(subset=['eW'])

In [ ]:
welfare = welfare.assign(ewg = np.where(welfare['eW'] <= 7, '취업률',
                               np.where(welfare['eW'] == 8 , '실업률','비경제')))
welfare['ewg'].value_counts()

In [ ]:
n_Employment =welfare.groupby('ewg',as_index=False)\
                 .agg(n= ('ewg','count'))
n_Employment

In [ ]:
sns.barplot(data = n_Employment, x='ewg', y='n')


In [ ]:
age_eW = welfare.groupby('ageg',as_index=False)\
                 ['ewg']\
                 .value_counts(normalize=True)
age_eW

In [ ]:

plt.title('23년 연령에 따른 경제활동 참여상태(취업,실업,비경제)')
sns.barplot(data=age_eW, y='ewg', x='proportion',hue='ageg')



In [ ]:
pivot_df=\
    age_eW[['ewg','ageg','proportion']].pivot(index='ewg',
                                              columns='ageg',
                                              values='proportion')
pivot_df

In [ ]:
pivot_df.plot.barh(stacked=True)

In [ ]:
welfare['no_eW'].dtype
welfare['no_eW'].value_counts() # 빈도수 

In [ ]:
#이상치 결측 처리 

welfare['no_eW']=np.where(welfare['no_eW'] == 99, np.nan, welfare['no_eW'])


# 결측 확인
print("결측 확인:", welfare['no_eW'].isna().sum()) 
welfare['no_eW'].isna().sum()

In [ ]:
#welfare=welfare.dropna(subset=['no_eW'])

In [ ]:
welfare_19['eW'].describe()

In [ ]:
welfare_19['eW'].dtype
welfare_19['eW'].value_counts() # 빈도수 

In [ ]:
#이상치 결측 처리 

welfare_19['eW']=np.where(welfare_19['eW'] == 99, np.nan, welfare_19['eW'])


# 결측 확인
print("결측 확인:", welfare_19['eW'].isna().sum()) 
welfare_19['eW'].isna().sum()

[23년도] '성별'과 '국민연금 납부여부' 항목 결측치 확인 및 대체

In [ ]:
welfare['sex'].value_counts()

In [ ]:
# 결측확인
welfare['sex'].isna().sum()

In [ ]:
# 성별, 항목 이름 부여
welfare['sex']=np.where(welfare['sex']==1,'male','female')

# 빈도 구하기
welfare['sex'].value_counts() # 빈도수

In [ ]:
welfare['pension_status'].value_counts('1')

In [ ]:
welfare['pension_status'].isna().sum()

In [ ]:
# 결측치의 개수 계산
missing_count = welfare['pension_status'].isna().sum()

# 각 값의 개수 계산
count_1 = int(missing_count * 0.8)  # 80%는 1로
count_2 = missing_count - count_1   # 나머지 20%는 2로

# 결측치를 채울 값 생성
fill_values = [1] * count_1 + [2] * count_2

# 결측치 위치의 인덱스 찾기
nan_indices = welfare[welfare['pension_status'].isna()].index

# 랜덤하게 결측치 인덱스를 선택하여 채우기
np.random.shuffle(fill_values)
welfare.loc[nan_indices, 'pension_status'] = fill_values

# 결과 확인
print("Pension Status Distribution After Filling:")
print(welfare['pension_status'].value_counts())

In [ ]:
# 결측치 개수 0 확인
welfare['pension_status'].isna().sum()

In [ ]:
# 국민연금 납부 여부 항목 이름 부여
welfare['pension_status']=np.where(welfare['pension_status']==1,'paid','unpaid')

[23년도] 성별에 따른 국민연금 납부 여부

In [ ]:
# 데이터 그룹화 및 집계
grouped_data = welfare.groupby(['sex', 'pension_status']).size().unstack()
               

# 막대 그래프 생성
grouped_data.plot(kind='bar', stacked=True, colormap='Set2')
plt.title('Pension Status by Sex')
plt.xlabel('Sex')
plt.ylabel('Count')
plt.legend(title='Pension Status')
plt.xticks(ticks=range(len(grouped_data.index)), labels=['Male', 'Female'], rotation=0)
plt.show()

[23년도] '장애종류'와 '외래진료 횟수' 항목 결측치 확인 및 대체 및 제거

In [ ]:
welfare['disabled_type'].isna().sum()

In [ ]:
welfare['disabled_type'].value_counts()

In [ ]:
# 0:비장애인, 1:지체장애, 2:뇌병변장애, 3:시각장애, 4:청각장애 5~: 그 외

welfare['disabled']=np.where(welfare['disabled_type']==0, '비장애인',
                           np.where(welfare['disabled_type']==1,'지체장애',
                                    np.where(welfare['disabled_type']==2,'뇌병변장애',
                                             np.where(welfare['disabled_type']==3,'시각장애',
                                                      np.where(welfare['disabled_type']==4,'청각장애','그 외')))))

welfare['disabled'].head(10)

In [ ]:
welfare['hospital_year'].isna().sum()

In [ ]:
# 외래진료횟수 150회 이상은 결측치로 처리
welfare['hospital_year']=np.where(welfare['hospital_year']>=150,np.nan,welfare['hospital_year'])

In [ ]:
# 결측치 제거
welfare = welfare.dropna(subset=['hospital_year'])

[23년도] 장애종류에 따른 평균 외래진료 횟수

In [ ]:
# 'disabled' 열의 각 범주별로 'hospital_year' 평균 계산
average_hospital_visits = welfare.groupby('disabled')['hospital_year'].mean().reset_index()

# 막대 그래프 생성
plt.figure(figsize=(10, 8))
sns.barplot(data=average_hospital_visits, x='disabled', y='hospital_year', palette='viridis', order=['비장애인','지체장애','뇌병변장애','시각장애','청각장애','그 외'])
plt.title('장애종류에 따른 평균 외래진료 횟수')
plt.xlabel('장애종류')
plt.ylabel('평균 외래진료 횟수')
plt.xticks(rotation=0)  # x축 레이블을 읽기 쉽게 회전
plt.show()

[23년도] '월급' 항목 결측치 확인 및 대체 및 제거

In [ ]:
# 이상치는 결측 처리
welfare['income']=np.where(welfare['income']>=9500, np.nan, welfare['income'])

# 결측치 확인
welfare['income'].isna().sum()

In [ ]:
# 장애종류별 월급 평균
dis_income=welfare.dropna(subset=['income'])\
                  .groupby('disabled',as_index=False)\
                  .agg(mean_income=('income','mean'))
dis_income

[23년도] 장애종류별 평균 월급

In [ ]:
# 'disabled' 컬럼을 범주형으로 변환하고 순서 지정
dis_income['disabled'] = pd.Categorical(dis_income['disabled'], 
                                       categories=['비장애인', '지체장애', '뇌병변장애', '시각장애', '청각장애', '그 외'],
                                       ordered=True)

# 데이터 정렬
dis_income = dis_income.sort_values('disabled')


# 선 그래프 그리기
sns.lineplot(data=dis_income, x='disabled', y='mean_income', marker='h', color='red')
plt.title("장애종류별 평균 월급")
plt.xlabel('장애종류')
plt.ylabel('평균수입')
plt.xticks(rotation=0) # x축 범주들 글자 각도 조정, 필요에 따라 조절
plt.show()

[23년도] 장애인 취업률 구하기

In [ ]:
welfare['eW'].isna().sum()

In [ ]:
welfare = welfare.dropna(subset=['eW'])

In [ ]:
welfare['eW'].value_counts()

In [ ]:
welfare['eW'] = np.where((welfare['eW'] == 8) | (welfare['eW'] == 9), '비근로자', '근로자')

In [ ]:
welfare['eW'].value_counts('1')

In [ ]:
# 장애 종류의 순서를 지정하여 데이터 정렬
order = ['비장애인', '지체장애', '뇌병변장애', '시각장애', '청각장애', '그 외']
welfare['disabled'] = pd.Categorical(welfare['disabled'], categories=order, ordered=True)

# 데이터 그룹화 및 집계
grouped_data = welfare.groupby(['disabled', 'eW']).size().unstack().reindex(columns=['근로자', '비근로자'], fill_value=0)

# 막대 그래프 생성
plt.figure(figsize=(12, 8))
ax = grouped_data.plot(kind='bar', stacked=True, colormap='Set1', ax=plt.gca())

# 제목 및 축 레이블 설정
plt.title('Disability Type and Employment Status')
plt.xlabel('Disability Type')
plt.ylabel('Count')

# x축 레이블 설정
plt.xticks(ticks=range(len(grouped_data.index)), labels=grouped_data.index, rotation=45)

# 범례 설정
plt.legend(title='Employment Status')

# 그래프 표시
plt.show()

In [ ]:

# 장애 종류의 순서를 지정하여 데이터 정렬
order = ['비장애인', '지체장애', '뇌병변장애', '시각장애', '청각장애', '그 외']
welfare['disabled'] = pd.Categorical(welfare['disabled'], categories=order, ordered=True)

# 데이터 그룹화 및 집계
grouped_data = welfare.groupby(['disabled', 'eW']).size().unstack().reindex(columns=['근로자', '비근로자'], fill_value=0)

# 비장애인 제외
filtered_data = grouped_data.drop(index='비장애인')

# 비장애인을 제외한 나머지 범주의 총 수
total_counts_excluding_non_disabled = filtered_data.sum().sum()

# 비장애인을 제외한 나머지 범주에서 근로자의 총 수
total_employed_excluding_non_disabled = filtered_data['근로자'].sum()

# 비율 계산
employment_percentage = (total_employed_excluding_non_disabled / total_counts_excluding_non_disabled) * 100

# 결과 출력
print(f'장애인 중에서 근로자 비율: {employment_percentage:.2f}%')
print(f'장애인 중에서 비근로자 비율: {100-employment_percentage:.2f}%')

In [ ]:
# 장애에 따른 국민 연금 
# 장애 종류의 순서를 지정하여 데이터 정렬
order = ['비장애인', '지체장애', '뇌병변장애', '시각장애', '청각장애', '그 외']
welfare['disabled'] = pd.Categorical(welfare['disabled'], categories=order, ordered=True)

# 데이터 그룹화 및 집계
grouped_data = welfare.groupby(['disabled', 'pension_status']).size().unstack().reindex(columns=['', ''], fill_value=0)

# 막대 그래프 생성
plt.figure(figsize=(12, 8))
ax = grouped_data.plot(kind='bar', stacked=True, colormap='Set1', ax=plt.gca())

# 제목 및 축 레이블 설정
plt.title('Disability Type and Employment Status')
plt.xlabel('Disability Type')
plt.ylabel('Count')

# x축 레이블 설정
plt.xticks(ticks=range(len(grouped_data.index)), labels=grouped_data.index, rotation=45)

# 범례 설정
plt.legend(title='Employment Status')

# 그래프 표시
plt.show()

In [ ]:
welfare_19['sex'].value_counts()

In [ ]:
# 결측확인
welfare_19['sex'].isna().sum()

In [ ]:
# 성별, 항목 이름 부여
welfare_19['sex']=np.where(welfare_19['sex']==1,'male','female')

# 빈도 구하기
welfare_19['sex'].value_counts() # 빈도수

In [ ]:
welfare_19['pension_status'].value_counts('1')

In [ ]:
welfare_19['pension_status'].isna().sum()

In [ ]:
# 결측치의 개수 계산
missing_count = welfare_19['pension_status'].isna().sum()

# 각 값의 개수 계산
count_1 = int(missing_count * 0.8)  # 80%는 1로
count_2 = missing_count - count_1   # 나머지 20%는 2로

# 결측치를 채울 값 생성
fill_values = [1] * count_1 + [2] * count_2

# 결측치 위치의 인덱스 찾기
nan_indices = welfare_19[welfare_19['pension_status'].isna()].index

# 랜덤하게 결측치 인덱스를 선택하여 채우기
np.random.shuffle(fill_values)
welfare_19.loc[nan_indices, 'pension_status'] = fill_values

# 결과 확인
print("Pension Status Distribution After Filling:")
print(welfare_19['pension_status'].value_counts())

In [ ]:
# 결측치 개수 0 확인
welfare_19['pension_status'].isna().sum()

In [ ]:
# 국민연금 납부 여부 항목 이름 부여
welfare_19['pension_status']=np.where(welfare_19['pension_status']==1,'paid','unpaid')

성별에 따른 국민연금 납부 여부

In [ ]:
# 데이터 그룹화 및 집계
grouped_data = welfare_19.groupby(['sex', 'pension_status']).size().unstack()

# 막대 그래프 생성
grouped_data.plot(kind='bar', stacked=True, colormap='Set2')
plt.title('Pension Status by Sex')
plt.xlabel('Sex')
plt.ylabel('Count')
plt.legend(title='Pension Status')
plt.xticks(ticks=range(len(grouped_data.index)), labels=['Male', 'Female'], rotation=0)
plt.show()

'장애종류'와 '외래진료 횟수' 항목 결측치 확인 및 대체 및 제거


In [ ]:
welfare_19['disabled_type'].isna().sum()

In [ ]:
welfare_19['disabled_type'].value_counts()

In [ ]:
# 0:비장애인, 1:지체장애, 2:뇌병변장애, 3:시각장애, 4:청각장애 5~: 그 외

welfare_19['disabled']=np.where(welfare_19['disabled_type']==0, '비장애인',
                           np.where(welfare_19['disabled_type']==1,'지체장애',
                                    np.where(welfare_19['disabled_type']==2,'뇌병변장애',
                                             np.where(welfare_19['disabled_type']==3,'시각장애',
                                                      np.where(welfare_19['disabled_type']==4,'청각장애','그 외')))))

welfare_19['disabled'].head(10)

In [ ]:
welfare_19['hospital_year'].isna().sum()

In [ ]:
# 외래진료횟수 150회 이상은 결측치로 처리
welfare_19['hospital_year']=np.where(welfare_19['hospital_year']>=150,np.nan,welfare_19['hospital_year'])

In [ ]:
# 결측치 제거
welfare_19 = welfare_19.dropna(subset=['hospital_year'])

장애종류에 따른 평균 외래진료 횟수

In [ ]:
# 'disabled' 열의 각 범주별로 'hospital_year' 평균 계산
average_hospital_visits = welfare_19.groupby('disabled')['hospital_year'].mean().reset_index()

# 막대 그래프 생성
plt.figure(figsize=(10, 8))
sns.barplot(data=average_hospital_visits, x='disabled', y='hospital_year', palette='viridis', order=['비장애인','지체장애','뇌병변장애','시각장애','청각장애','그 외'])
plt.title('장애종류에 따른 평균 외래진료 횟수')
plt.xlabel('장애종류')
plt.ylabel('평균 외래진료 횟수')
plt.xticks(rotation=0)  # x축 레이블을 읽기 쉽게 회전
plt.show()

'월급' 항목 결측치 확인 및 대체 및 제거


In [ ]:
# 이상치는 결측 처리
welfare_19['income']=np.where(welfare_19['income']>=9500, np.nan, welfare_19['income'])

# 결측치 확인
welfare_19['income'].isna().sum()

In [ ]:
# 장애종류별 월급 평균
dis_income=welfare_19.dropna(subset=['income'])\
                  .groupby('disabled',as_index=False)\
                  .agg(mean_income=('income','mean'))
dis_income

장애종류별 평균 월급

In [ ]:
# 'disabled' 컬럼을 범주형으로 변환하고 순서 지정
dis_income['disabled'] = pd.Categorical(dis_income['disabled'], 
                                       categories=['비장애인', '지체장애', '뇌병변장애', '시각장애', '청각장애', '그 외'],
                                       ordered=True)

# 데이터 정렬
dis_income = dis_income.sort_values('disabled')


# 선 그래프 그리기
sns.lineplot(data=dis_income, x='disabled', y='mean_income', marker='h', color='red')
plt.title("장애종류별 평균 월급")
plt.xlabel('장애종류')
plt.ylabel('평균수입')
plt.xticks(rotation=0) # x축 범주들 글자 각도 조정, 필요에 따라 조절
plt.show()

In [ ]:
welfare_19['eW'].isna().sum()

In [ ]:
welfare_19 = welfare_19.dropna(subset=['eW'])

In [ ]:
welfare_19['eW'].value_counts()

In [ ]:
welfare_19['eW'] = np.where((welfare_19['eW'] == 8) | (welfare_19['eW'] == 9), '비근로자', '근로자')

In [ ]:
welfare_19['eW'].value_counts('1')

In [ ]:
# 장애 종류의 순서를 지정하여 데이터 정렬
order = ['비장애인', '지체장애', '뇌병변장애', '시각장애', '청각장애', '그 외']
welfare_19['disabled'] = pd.Categorical(welfare_19['disabled'], categories=order, ordered=True)

# 데이터 그룹화 및 집계
grouped_data = welfare_19.groupby(['disabled', 'eW']).size().unstack().reindex(columns=['근로자', '비근로자'], fill_value=0)

# 막대 그래프 생성
plt.figure(figsize=(12, 8))
ax = grouped_data.plot(kind='bar', stacked=True, colormap='Set1', ax=plt.gca())

# 제목 및 축 레이블 설정
plt.title('Disability Type and Employment Status')
plt.xlabel('Disability Type')
plt.ylabel('Count')

# x축 레이블 설정
plt.xticks(ticks=range(len(grouped_data.index)), labels=grouped_data.index, rotation=45)

# 범례 설정
plt.legend(title='Employment Status')

# 그래프 표시
plt.show()

In [ ]:

# 장애 종류의 순서를 지정하여 데이터 정렬
order = ['비장애인', '지체장애', '뇌병변장애', '시각장애', '청각장애', '그 외']
welfare_19['disabled'] = pd.Categorical(welfare_19['disabled'], categories=order, ordered=True)

# 데이터 그룹화 및 집계
grouped_data = welfare_19.groupby(['disabled', 'eW']).size().unstack().reindex(columns=['근로자', '비근로자'], fill_value=0)

# 비장애인 제외
filtered_data = grouped_data.drop(index='비장애인')

# 비장애인을 제외한 나머지 범주의 총 수
total_counts_excluding_non_disabled = filtered_data.sum().sum()

# 비장애인을 제외한 나머지 범주에서 근로자의 총 수
total_employed_excluding_non_disabled = filtered_data['근로자'].sum()

# 비율 계산
employment_percentage = (total_employed_excluding_non_disabled / total_counts_excluding_non_disabled) * 100

# 결과 출력
print(f'장애인 중에서 근로자 비율: {employment_percentage:.2f}%')
print(f'장애인 중에서 비근로자 비율: {100-employment_percentage:.2f}%')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 예시 데이터프레임 (실제 데이터프레임을 사용해 주세요)
data = {
    'disabled': ['비장애인', '지체장애', '뇌병변장애', '시각장애', '청각장애', '그 외']*10,
    'hospital_year': [5, 7, 6, 4, 3, 8]*10
}
welfare_19 = pd.DataFrame(data)

# 각 범주별 평균 계산
average_hospital_visits = welfare_19.groupby('disabled')['hospital_year'].mean().reset_index()

# 두 개의 그래프를 같은 플롯에 그리기
fig, ax = plt.subplots(1, 2, figsize=(20, 8), sharey=True)

# 첫 번째 그래프
sns.barplot(data=average_hospital_visits, x='disabled', y='hospital_year', palette='viridis', order=['비장애인','지체장애','뇌병변장애','시각장애','청각장애','그 외'], ax=ax[0])
ax[0].set_title('장애종류에 따른 평균 외래진료 횟수 (첫 번째 그래프)')
ax[0].set_xlabel('장애종류')
ax[0].set_ylabel('평균 외래진료 횟수')
ax[0].tick_params(axis='x', rotation=45)  # x축 레이블을 읽기 쉽게 회전

# 두 번째 그래프 (같은 데이터로 다르게 표현할 수 있음)
sns.barplot(data=average_hospital_visits, x='disabled', y='hospital_year', palette='magma', order=['비장애인','지체장애','뇌병변장애','시각장애','청각장애','그 외'], ax=ax[1])
ax[1].set_title('장애종류에 따른 평균 외래진료 횟수 (두 번째 그래프)')
ax[1].set_xlabel('장애종류')
ax[1].set_ylabel('평균 외래진료 횟수')
ax[1].tick_params(axis='x', rotation=45)  # x축 레이블을 읽기 쉽게 회전

# 플롯 간 간격 조정
plt.tight_layout()
plt.show()



In [ ]:
# student: 25세이하, young: 26~29세, middle: 30~59세, old: 나머지

welfare=welfare.assign(ages=np.where(welfare['age']<=25, 'student',
                                     np.where(welfare['age']<30, 'young',
                                              np.where(welfare['age']<=59, 'middle','old'))))
welfare=welfare['ages'].value_counts()


In [ ]:
# 비경제활동사유 항목에 이름 붙이기

welfare = welfare.assign(no_eWs=np.where(welfare['no_eW'] == 1, '근로무능력',
                                     np.where(welfare['no_eW'] == 2, '군복무',
                                         np.where(welfare['no_eW'] == 3, '학업',
                                            np.where(welfare['no_eW'] == 4, '진학준비',
                                               np.where(welfare['no_eW'] == 5, '취업준비',
                                                  np.where(welfare['no_eW'] == 6, '가사',
                                                     np.where(welfare['no_eW'] == 7, '양육',
                                                        np.where(welfare['no_eW'] == 8, '간병',
                                                           np.where(welfare['no_eW'] == 9, '구직활동포기',
                                                              np.where(welfare['no_eW'] == 10, '근로의사없음', '기타')
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    )
)


In [ ]:
# x축 순서 설정
order = ['student', 'young', 'middle', 'old']

# seaborn을 사용하여 연령대별 비경제활동사유를 시각화
plt.figure(figsize=(10, 6))
ax = sns.countplot(data=welfare, x='ages', hue='no_eWs', palette='Set2', order=order)

# y축을 로그 스케일로 변환 (낮은 수치도 잘 보이도록)
ax.set_yscale('log')

# 막대 위에 실제 수치 표시하기
for p in ax.patches:
    height = p.get_height()
    # 막대의 높이를 로그 스케일로 변환된 값을 이용하여 표시
    if height > 0:  # 0인 경우는 제외
        ax.text(p.get_x() + p.get_width() / 2., height, f'{int(height):,}', ha="center", va="center", fontsize=10, color='black')

# 제목과 레이블 설정
plt.title('연령대별 비경제활동사유')
plt.xlabel('연령대')
plt.ylabel('Count (Log Scale)')

# 범례 설정: 글꼴 크기 조정
plt.legend(title='비경제활동사유', fontsize='small', title_fontsize='13', loc='upper left', bbox_to_anchor=(1, 1))

plt.xticks(rotation=0)
plt.tight_layout()  # 레이아웃 조정하여 범례가 잘리지 않도록 함
plt.show()